In [14]:
import pandas as pd
train_data = pd.read_csv('./DataSets/train_modified.csv')
train_data.head()

,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0,0.0,ID000002C20,300000,5,20000,1,0,37,1,...,0,0,0,0,1,1,0,1,0,0
1,0,0.0,ID000004E40,200000,2,35000,3,13,30,0,...,0,0,0,0,1,0,1,1,0,0
2,0,0.0,ID000007H20,600000,4,22500,1,0,34,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0.0,ID000008I30,1000000,5,35000,3,10,28,1,...,0,0,0,0,0,0,1,0,0,1
4,0,25000.0,ID000009J40,500000,2,100000,3,17,31,1,...,0,0,0,0,0,0,1,0,0,1


In [31]:
x_columns = [x for x in train_data.columns if x not in ['ID', 'Disbursed']]
x = train_data[x_columns]
y = train_data['Disbursed']
print(x.shape)
print(y.shape)

(20000, 49)
(20000,)


In [33]:
#使用默认参数进行训练
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
gbm1 = GradientBoostingClassifier()
gbm1.fit(x,y)
y_pred = gbm1.predict(x)
print(metrics.accuracy_score(y_pred,y))

0.98525


In [42]:
#使用网格搜索选择最佳迭代次数和惩罚系数
from sklearn.model_selection import GridSearchCV
gscv1 = GridSearchCV(estimator=GradientBoostingClassifier(),
                     param_grid={'n_estimators':[50,100,150,200],'learning_rate':[0.1,0.3,0.4]})
gscv1.fit(x,y)
print(gscv1.best_score_,gscv1.best_params_)
gscv1.grid_scores_

0.98315 {'learning_rate': 0.1, 'n_estimators': 50}


[mean: 0.98315, std: 0.00049, params: {'learning_rate': 0.1, 'n_estimators': 50},
 mean: 0.98275, std: 0.00049, params: {'learning_rate': 0.1, 'n_estimators': 100},
 mean: 0.98225, std: 0.00051, params: {'learning_rate': 0.1, 'n_estimators': 150},
 mean: 0.98240, std: 0.00039, params: {'learning_rate': 0.1, 'n_estimators': 200},
 mean: 0.98190, std: 0.00078, params: {'learning_rate': 0.3, 'n_estimators': 50},
 mean: 0.98095, std: 0.00068, params: {'learning_rate': 0.3, 'n_estimators': 100},
 mean: 0.98060, std: 0.00086, params: {'learning_rate': 0.3, 'n_estimators': 150},
 mean: 0.98045, std: 0.00092, params: {'learning_rate': 0.3, 'n_estimators': 200},
 mean: 0.98110, std: 0.00056, params: {'learning_rate': 0.4, 'n_estimators': 50},
 mean: 0.98045, std: 0.00067, params: {'learning_rate': 0.4, 'n_estimators': 100},
 mean: 0.97930, std: 0.00141, params: {'learning_rate': 0.4, 'n_estimators': 150},
 mean: 0.97890, std: 0.00163, params: {'learning_rate': 0.4, 'n_estimators': 200}]

In [43]:
#使用网格搜索选择最大深度和叶子节点划分阈值
gscv1 = GridSearchCV(estimator=GradientBoostingClassifier(n_estimators=50,learning_rate=0.1),
                     param_grid={'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)})
gscv1.fit(x,y)
print(gscv1.best_score_,gscv1.best_params_)
gscv1.grid_scores_

0.9838 {'max_depth': 5, 'min_samples_split': 500}


[mean: 0.98345, std: 0.00049, params: {'max_depth': 3, 'min_samples_split': 100},
 mean: 0.98340, std: 0.00058, params: {'max_depth': 3, 'min_samples_split': 300},
 mean: 0.98360, std: 0.00028, params: {'max_depth': 3, 'min_samples_split': 500},
 mean: 0.98375, std: 0.00028, params: {'max_depth': 3, 'min_samples_split': 700},
 mean: 0.98295, std: 0.00086, params: {'max_depth': 5, 'min_samples_split': 100},
 mean: 0.98370, std: 0.00019, params: {'max_depth': 5, 'min_samples_split': 300},
 mean: 0.98380, std: 0.00012, params: {'max_depth': 5, 'min_samples_split': 500},
 mean: 0.98370, std: 0.00014, params: {'max_depth': 5, 'min_samples_split': 700},
 mean: 0.98300, std: 0.00107, params: {'max_depth': 7, 'min_samples_split': 100},
 mean: 0.98350, std: 0.00021, params: {'max_depth': 7, 'min_samples_split': 300},
 mean: 0.98360, std: 0.00014, params: {'max_depth': 7, 'min_samples_split': 500},
 mean: 0.98355, std: 0.00019, params: {'max_depth': 7, 'min_samples_split': 700},
 mean: 0.98265, 

In [40]:
#使用上述选择的参数训练新网络
gbm1 = GradientBoostingClassifier()
gbm1.fit(x,y)
y_pred = gbm1.predict(x)
print(metrics.accuracy_score(y_pred,y))

0.9833 {'n_estimators': 30}
